In [ ]:
#import dependencies
from random import choice
import random
import heapq
import pandas as pd
import numpy as np
import math
from deap import base, creator, tools, algorithms
from plot_utils import *
from collections import defaultdict, deque, Counter
import time

#read data
position_data = pd.read_csv("sub_data_file_with_header.csv")
#position_data.head()

# #add the base station to the dataframe if not added
# new_rows = pd.DataFrame({'Nos.': [len(position_data) + 1, len(position_data) + 2],
#                          'x': [5000, -5000],
#                          'y': [-5000, 5000]})
# position_data = pd.concat([position_data, new_rows], ignore_index=True)

#extract the index, x, y
i = position_data.loc[:, 'No.']
x = position_data.loc[:,'x']
y = position_data.loc[:,'y']

#define function for distance and transmission rate

def distance(x1, x2, y1, y2):
    return math.sqrt(((x2 - x1) ** 2) + ((y2 - y1) ** 2))

def transmission_rate(d):
    if d >= 3000:
        return 0
    elif d >= 2500 and d <3000:
        return 1
    elif d >= 2000 and d < 2500:
        return 2
    elif d>= 1500 and d < 2000:
        return 3
    elif d >= 1000 and d < 1500:
        return 4
    elif d >= 500 and d < 1000:
        return 5
    elif d >0 and d< 500:
        return 7
    else:
        return 0

#generate the graph using the predefined functions
graph = {}
for r,x_1,y_1 in zip(i,x,y):
    graph["Node_" + str(r)] = {}
    for j, x_2,y_2 in zip(i,x,y):
        #if statement to chech if x1,y1,x2,y2 are the same, and continue
        if (x_1 == x_2) and (y_1 == y_2):
            continue
        #call the distance function, returns transmission rate
        dist = distance(x_1, x_2, y_1, y_2)
        tr_rate = transmission_rate(dist)
        #check if the transmission is greater than zero
        if tr_rate > 0:
            graph["Node_" + str(r)]["Node_" + str(j)] = tr_rate


#Helper functions
#Helps get transmission rate in a path
def get_tran_rate(bp):
    rates = []
    for i in range(len(bp)-1):
        rates.append(graph[bp[i]][bp[i+1]])
    return min(rates)

#Helps get row number to make plots easier
def extract_point(bp):
    all_value = []
    for i in bp:
        all_value.append(int(i.split("_")[1]) -1)
    return all_value

In [ ]:
import heapq

# Constants
max_trans_rate = 7
max_latency = 1650
alpha = 0.3
beta = 0.7
# Updated cost_function
def dj_cost_function(rate, latency, alpha, beta):
    normalized_latency = latency / max_latency
    score = alpha * rate - beta * normalized_latency
    #print(f"dj_cost_function - Rate: {rate}, Latency: {latency}, Score: {score}")
    return score

In [ ]:
def dijkstra_with_tracking(start, target, alpha, beta):
    # Initialize tracking lists
    tracked_latency = []
    tracked_rate = []

    # Initialize Dijkstra's structures
    max_min_rate = {node: 0 for node in graph}
    max_min_rate[start] = max_trans_rate
    heap_start = [(-max_min_rate[start], start, [start])]
    best_path = []
    best_min_score = 0

    while heap_start:
        neg_rate, node, path = heapq.heappop(heap_start)
        rate = -neg_rate

        # Debug: Current processing state
        #print(f"Processing node: {node}, Rate: {rate}, Path: {path}")

        # Track the current path's latency and rate
        latency = (len(path) - 1) * 30 if len(path) > 1 else 0  # Current path latency
        tracked_latency.append(latency)
        tracked_rate.append(rate)

        if node in target:
            #print(f"Target node {node} reached. Current path: {path}, Rate: {rate}")
            if rate > best_min_score:
                #print(f"Updating best path to {path}")
                best_min_score = rate
                best_path = path
            continue

        for next_node, next_rate in graph[node].items():
            # Calculate latency and score
            new_latency = latency + 30
            next_opt_score = dj_cost_function(next_rate, new_latency, alpha, beta)
            update_rate = min(rate, next_opt_score)

            #print(f"Checking neighbor {next_node}. Rate: {next_rate}, New score: {next_opt_score}, Updated rate: {update_rate}")

            if update_rate > max_min_rate[next_node]:
                max_min_rate[next_node] = update_rate
                #print(f"Pushing to heap: {next_node}, Update rate: {update_rate}, Path: {path + [next_node]}")
                heapq.heappush(heap_start, (-update_rate, next_node, path + [next_node]))

                #print(f"Skipping neighbor {next_node}, Update rate: {update_rate}, Current max_min_rate: {max_min_rate[next_node]}")

    # Return the final best path, latency, rate, and tracking data
    if not best_path:
        print("No path found to target.")
        return [], float('inf'), 0, tracked_latency, tracked_rate
    else:
        final_latency = (len(best_path) - 1) * 30
        final_rate = get_tran_rate(best_path)
        #print(f"Best path found: {best_path}, Latency: {final_latency}, Rate: {final_rate}")
        return best_path, final_latency, final_rate, tracked_latency, tracked_rate


In [ ]:
# Evaluate each node separately to get the optimised alpha and beta and get the alpha beta with maximum occurence.
def evaluate_each_node(graph, nodes, base_stations):
    node_results = {}
    alpha_beta_pairs = []

    for node in nodes:
        best_alpha = 0
        best_beta = 0
        best_score = float('-inf')
        best_path = []

        for alpha in np.arange(0.1, 1.0, 0.1):
            for beta in np.arange(0.1, 1.0, 0.1):
                if round(alpha + beta, 1) != 1:  # Ensure alpha + beta = 1
                    continue

                paths_and_scores = []
                path, score, _,_,_ = dijkstra_with_tracking(node, target, alpha, beta)
                paths_and_scores.append((path, score))

                current_best_path, current_best_score = max(paths_and_scores, key=lambda x: x[1])

                if current_best_score > best_score:
                    best_score = current_best_score
                    best_alpha = alpha
                    best_beta = beta
                    best_path = current_best_path

        # Store the best alpha and beta for this node
        alpha_beta_pairs.append((best_alpha, best_beta))
        node_results[node] = {
            "best_alpha": best_alpha,
            "best_beta": best_beta,
            "best_path": best_path,
            "best_score": best_score,
        }

    # Determine the mode of alpha and beta pairs
    alpha_mode = Counter([pair[0] for pair in alpha_beta_pairs]).most_common(1)[0][0]
    beta_mode = Counter([pair[1] for pair in alpha_beta_pairs]).most_common(1)[0][0]

    # Normalize the mode values to ensure their sum equals 1
    total_mode = alpha_mode + beta_mode
    alpha_mode_normalized = alpha_mode / total_mode
    beta_mode_normalized = beta_mode / total_mode

    return node_results, alpha_mode_normalized, beta_mode_normalized


# Get all keys from the graph
keys_list = list(graph.keys())

# Remove 'Node_1' and 'Node_152' from the list if they exist
nodes = [key for key in keys_list if key not in ['Node_1', 'Node_152']]
target = ['Node_1', 'Node_152']

# Run the evaluation
results, alpha_mode, beta_mode = evaluate_each_node(graph, nodes, target)

# Display the results
print(f"Normalized alpha mode: {alpha_mode:.2f}")
print(f"Normalized beta mode: {beta_mode:.2f}\n")



Normalized alpha mode: 0.10
Normalized beta mode: 0.90

